In [16]:
import pandas as pd
from langgraph.graph import StateGraph
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage
from langchain_core.language_models import BaseChatModel
from langgraph.checkpoint.sqlite import SqliteSaver
from langchain_google_genai import ChatGoogleGenerativeAI
from typing import List, TypedDict
from openpyxl.utils import get_column_letter
from openpyxl.styles import Alignment
import re
import pdfplumber

# 定義狀態字典類型，用於保存每個客戶的狀態信息
class CustomerState(TypedDict):
    news_content: str  # 輸入的新聞內容
    customer_features: List[str]  # 客戶特徵列表
    extracted_features: str  # 提取的客戶特徵摘要
    news_summary: str  # 基於客戶特徵的新聞摘要
    investment_advice_result: str  # 生成的投資建議
    impact_score: int  # 新增的影響程度分數

# 定義生成具體特徵的函數
def generate_customer_specific_features(row):
    investment_allocation = (
        f"投資配置:\n"
        f" - 近一年總成交量: {round(row['近一年總成交量'])}, 股票類型交易量: {round(row['股票類型交易量'])}, "
        f"債券類型交易量: {round(row['債券類型交易量'])}, 平衡類型交易量: {round(row['平衡類型交易量'])}, 貨幣類型交易量: {round(row['貨幣類型交易量'])}\n"
        f" - 近一年總成交次數: {round(row['近一年總成交次數'])}, 股票類型交易次數: {round(row['股票類型交易次數'])}, "
        f"債券類型交易次數: {round(row['債券類型交易次數'])}, 平衡類型交易次數: {round(row['平衡類型交易次數'])}, 貨幣類型交易次數: {round(row['貨幣類型交易次數'])}"
    )
    
    risk_preference = (
        f"風險偏好: 客戶風險等級: {row['客戶風險等級']}, RR1佔比: {round(row['RR1佔比'])}%, RR2佔比: {round(row['RR2佔比'])}%, "
        f"RR3佔比: {round(row['RR3佔比'])}%, RR4佔比: {round(row['RR4佔比'])}%, RR5佔比: {round(row['RR5佔比'])}%"
    )

    holdings_status = (
        f"持倉狀況: 庫存總量: {round(row['庫存總量'])}, 股票類型庫存量: {round(row['股票類型庫存量'])}, "
        f"債券類型庫存量: {round(row['債券類型庫存量'])}, 平衡類型庫存量: {round(row['平衡類型庫存量'])}, 貨幣類型庫存量: {round(row['貨幣類型庫存量'])}"
    )
    
    return f"{investment_allocation}\n{risk_preference}\n{holdings_status}"

# 定義格式化輸出結果的函數
def format_output_for_customer(customer_name, extracted_features, news_summary, investment_advice_result, impact_score):
    return (f"**客戶 {customer_name}**\n\n"
            f"{extracted_features}\n\n"  # 顯示具體數字的客戶特徵
            f"## LLM 生成的客戶特徵描述:\n{news_summary}\n\n"
            f"## 投資建議:\n{investment_advice_result}\n\n"
            f"新聞對客戶的影響程度: {impact_score} 分\n")

# 定義PDF提取摘要的函數
def extract_pdf_summary(pdf_file_path):
    summary = []
    with pdfplumber.open(pdf_file_path) as pdf:
        for page in pdf.pages:
            text = page.extract_text()
            summary.append(text[:200] + "...")  # 提取前200個字作為摘要
    return "\n".join(summary)

# 初始化模型和檢查點保存器
model = ChatGoogleGenerativeAI(model="gemini-1.5-pro", temperature=0)
memory = SqliteSaver.from_conn_string(":memory:")

bot = PersonalizedNewsAgent(model=model, checkpointer=memory)
    
thread = {"configurable": {"thread_id": "1"}}

# 讀取Excel表格
df = pd.read_excel("teste_3.xlsx")

# 從PDF提取新聞內容
pdf_summary = extract_pdf_summary("test.pdf")

# 使用 LLM 生成 PDF 的摘要
messages = [
    SystemMessage(content="請將以下PDF的內容生成條列摘要，請用繁體中文回答"),
    HumanMessage(content=pdf_summary)
]

# 調用模型生成PDF摘要
response = model.invoke(messages)
pdf_summary_list = response.content.strip()  # 生成的PDF摘要

# 打印生成的摘要列表，供參考
print("生成的PDF摘要:")
print(pdf_summary_list)

# 定義輸入的新聞內容，將生成的PDF摘要作為新聞內容
state = {
    "news_content": pdf_summary_list
}

# 生成客戶特徵
all_customers_output = []
all_customers_data = []

# 自動讀取Excel欄位名稱，排除ID欄位
selected_columns = df.columns[1:]

# 迭代每個客戶，生成特徵摘要和投資建議
for index, row in df.iterrows():
    customer_name = row['ID']
    
    # 生成客戶的具體特徵
    specific_features = generate_customer_specific_features(row)
    
    # 根據選擇的欄位來生成額外的客戶特徵，並使用LLM生成描述
    customer_features = []
    for col in selected_columns:
        if pd.notna(row[col]):  # 檢查該欄位是否有數值
            feature = f"{col}: {row[col]}"
            customer_features.append(feature)
    
    # 將具體特徵和LLM生成特徵合併
    state["customer_features"] = [specific_features] + customer_features

    # 執行圖形流程
    result = bot.graph.invoke(state, thread)

    # 檢查生成的投資建議是否為空，如果是空的，給予警告並記錄
    if not result["investment_advice_result"]:
        print(f"警告: {customer_name} 的投資建議生成為空！")

    # 格式化輸出結果
    customer_output = format_output_for_customer(
        customer_name, 
        specific_features,  # 使用具體特徵顯示具體數字
        result["news_summary"],  # LLM生成的客戶特徵描述
        result["investment_advice_result"], 
        result["impact_score"]  # 添加影響程度分數
    )
    
    all_customers_output.append(customer_output)

    all_customers_data.append({
        "客戶": customer_name,
        "客戶特徵": specific_features,
        "新聞摘要": result["news_summary"],
        "投資建議": result["investment_advice_result"],
        "影響程度分數": result["impact_score"]  # 新增的影響程度分數列
    })

# 創建DataFrame並輸出到Excel
output_df = pd.DataFrame(all_customers_data)
with pd.ExcelWriter("Personalized_Investment_Advice_Output_V3.xlsx", engine="openpyxl") as writer:
    output_df.to_excel(writer, sheet_name="Sheet1", index=False)
    worksheet = writer.sheets["Sheet1"]
    
    # 調整欄位寬度
    for i, col in enumerate(output_df.columns):
        max_width = max(output_df[col].astype(str).map(len).max(), len(col)) + 2
        worksheet.column_dimensions[get_column_letter(i + 1)].width = max_width
    
    # 文字換行
    for row in worksheet.iter_rows():
        for cell in row:
            cell.alignment = Alignment(wrap_text=True)

print("Excel 文件已生成並保存，請檢查結果。")


生成的PDF摘要:
## 投資策略週報 (2024/09/02 – 2024/09/06)  條列摘要

**一、 投資策略**

* **債券市場:**
    * 增持金融債，增加存續期間：預期聯準會降息後殖利率將下行，金融債等景氣敏感產業將受惠於利差收斂。
    * 短期美債殖利率再下行空間有限：十年期殖利率與政策利率差值已擴大至歷史低位。
    * 預防性降息有利循環性產業利差收斂：降息將為循環性產業的基本面動能提供支撐。
    * 投等債、非投等債利差有望持續收斂：市場情緒正在改善，非投等債利差持續向下收斂。

* **股票市場:**
    * 股市仍有震盪風險，但可開始留意投資風格轉換。
    * 科技股短期將延續修正行情：Nvidia 財報不如預期，資金更為偏好價值股。
    * 短期雖仍有波動，但降息將為股市中期提供支撐：過去預防性降息後，股市後6個月平均上漲15%。
    * 9月選舉添加股市不確定性：總統大選辯論將至，市場觀望情緒濃厚。

* **匯市展望:**
    * 日圓重新轉強：BoJ 總裁鷹派發言，疊加市場過度樂觀聯準會降息空間，導致美元指數近期疲態。
    * 美元指數或迎來短期反彈：美元指數與美債殖利率的下跌，似乎過度反應衰退風險與降息預期。

* **商品市場:**
    * 油價劇烈波動：利比亞可能暫停原油生產、出口，但長期影響有限。
    * 金價維持高檔震盪：美元短線跌幅過大，但預期將迎來反彈。

**二、 市場訊息**

* 利比亞將暫停原油生產，短期激勵油價走勢，但長期影響有限。
* NVIDIA 營收優於預期，但毛利率不及預期，盤後股價重挫。
* 7月台灣景氣燈號轉為黃紅燈，主因是受到颱風影響。
* 中國滬深300指數持續走弱，但金融股逆勢上漲，反應銀行業改善的利潤表現。

**三、 機會市場**

* **股票市場:**
    * 美國：9月總統大選辯論等不確定因素猶存，若股市再次下跌，可重新將焦點轉向成長股。
    * 中國：儘管近期滬深300持續走弱，但金融類股卻逆勢上漲，或暗示大盤走勢離落底不遠。

**四、 產品介紹**

* 9月推薦產品包括：SPDR 標普500、iShares MSCI 中國、iShares 20Y+ 美國公債、SPDR 黃金等。
* 基金策略：
    * 股